In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [3]:
import pickle 

In [4]:
# load the model , scaler , and the encoders
model=load_model('model.h5')

# load the encoder and the scaler
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

with open('le_gender.pkl','rb') as file:
    le_gender=pickle.load(file)

with open('One_geo_encoder.pkl','rb') as file:
    One_geo_encoder=pickle.load(file)

In [8]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [16]:
#one hot encoded geo
geo_encoded=One_geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=One_geo_encoder.get_feature_names_out(['Geography']))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [18]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df=pd.DataFrame([input_data])

In [22]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [24]:
input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [26]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [34]:
input_df['Gender']=le_gender.transform(input_df['Gender'])

In [36]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
input_df=scaler.transform(input_df)

In [40]:
input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [42]:
prediction = model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [52]:
prediction_proba=prediction[0][0]

In [54]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
